# Combining together the piecewise voltage simulation

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
import math
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu
import flopter.core.constants as c
import flopter.spice.inputparser as inp
import flopter.spice.normalise as nrm
import flopter.spice.utils as spu

In [2]:
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/coding/projects/flopter/flopter/core/lputils.py'>

# Preamble of necessary stuff

This bit creates all teh necessary variables for plotting to commence, including loading the dataset, creating a denormaliser, extracting relevant probe dimensions from an input file and defining a few functions for ease of manipulation. 

In [3]:
spice_dir = pth.Path('/home/jleland/data/external_big/spice/')
os.chdir(spice_dir)

In [4]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [5]:
cb_palette = ['#377eb8', '#ff7f00', '#4daf4a', 
              '#f781bf', '#a65628', '#984ea3', 
              '#999999', '#e41a1c', '#dede00',
              '#377eb8', '#ff7f00', '#4daf4a', 
              '#f781bf', '#a65628', '#984ea3', 
              '#999999', '#e41a1c', '#dede00']

In [6]:
skippable_scans = set([
    'marconi/spice2/piecewise/flat_flush_piecewise',
    'marconi/spice2/piecewise/flat_recessed_10',
    'marconi/spice2/piecewise/flat_recessed_10_1_',
    'marconi/spice2/piecewise/flat_recessed_8-12',
]) 
single_sims = set()

In [7]:
# scans_searchstr = '*/*/sheath_exp/*'
# scans_searchstr = '*/*/new_sheath_exp/*'
scans_searchstr = '*/*/piecewise/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'t', 'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 
                          'Npc', 'snumber', 'nproc', 'by', 'bz', 'rho01', 'rho02', 'rho'}
desired_variables = (td.DEFAULT_REDUCED_DATASET | non_standard_variables) - {td.OBJECTSCURRENTFLUXE, td.OBJECTSCURRENTFLUXI}
allowed_pots = [
    '0.0', '-1.0', '-2.0', '-3.0', '-4.0', '-5.0', '-6.0', '-7.0', 
    '-8.0', '-9.0', '-10.0', '-12.0', '-18.0', '-25.0'
]

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = [scan_run for scan_run in glob.glob(scan + angles_search_str) 
                              if scan_run.split('__pot_')[-1] in allowed_pots]



scans = list(scans)
scans.sort()

In [8]:
for i, scan in enumerate(scans):
    print(f"[{i}]: {scan}")
    for j, run in enumerate(all_run_dirs[scan]):
        print(f"\t[{i},{j}]: {'/'.join(run.split('/')[-2:])}")

[0]: marconi/spice2/piecewise/flat_flush
	[0,0]: flat_flush/alpha_yz_-3.0__pot_-1.0
	[0,1]: flat_flush/alpha_yz_-3.0__pot_-10.0
	[0,2]: flat_flush/alpha_yz_-3.0__pot_-12.0
	[0,3]: flat_flush/alpha_yz_-3.0__pot_-18.0
	[0,4]: flat_flush/alpha_yz_-3.0__pot_-2.0
	[0,5]: flat_flush/alpha_yz_-3.0__pot_-25.0
	[0,6]: flat_flush/alpha_yz_-3.0__pot_-3.0
	[0,7]: flat_flush/alpha_yz_-3.0__pot_-4.0
	[0,8]: flat_flush/alpha_yz_-3.0__pot_-5.0
	[0,9]: flat_flush/alpha_yz_-3.0__pot_-6.0
	[0,10]: flat_flush/alpha_yz_-3.0__pot_-7.0
	[0,11]: flat_flush/alpha_yz_-3.0__pot_-8.0
	[0,12]: flat_flush/alpha_yz_-3.0__pot_-9.0
	[0,13]: flat_flush/alpha_yz_-3.0__pot_0.0
	[0,14]: flat_flush/alpha_yz_-5.0__pot_-1.0
	[0,15]: flat_flush/alpha_yz_-5.0__pot_-10.0
	[0,16]: flat_flush/alpha_yz_-5.0__pot_-12.0
	[0,17]: flat_flush/alpha_yz_-5.0__pot_-18.0
	[0,18]: flat_flush/alpha_yz_-5.0__pot_-2.0
	[0,19]: flat_flush/alpha_yz_-5.0__pot_-25.0
	[0,20]: flat_flush/alpha_yz_-5.0__pot_-3.0
	[0,21]: flat_flush/alpha_yz_-5.0__pot

In [141]:
spl_path = spice_dir / all_run_dirs[scans[3]][0]
print(spl_path)

/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush_fixedpot_highV/alpha_yz_-10.0__pot_-18.0


## Start with the one

In [142]:
bups = list(spl_path.glob('backup*'))
bups.sort()
final_state_path = spl_path / bups[-1].name

In [143]:
splopter =  spl.Splopter(final_state_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True, check_voltage_error_fl=False)

splopter.prepare(find_se_temp_fl=False, homogenise_fl=False)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush_fixedpot_highV/alpha_yz_-10.0__pot_-18.0/backup_20201118-0318) doesn't seem to be valid.
Continuing anyway.


In [89]:
splopter.tdata.t_dict.keys()

dict_keys(['__header__', '__version__', '__globals__', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'dt', 'dz', 'nproc', 'q', 'm', 'Temp', 'rho', 'Pot', 'Potvac', 't', 'snumber', 'by', 'bz', 'npartproc', 'ProbePot', 'objectsenum', 'objectscurrenti', 'objectscurrente', 'rho01', 'rho02', 'alphayz', 'alphaxz', 'fvlimits', 'iHistTopx2', 'diaghistories', 'Esct', 'diagm', 'eHistSEx3', 'solw01', 'totalenergy', 'Np', 'tau', 'bx', 'surfacematrix', 'hpos', 'edgecharge', 'emmitdiag02', 'flagm', 'nodiagreg', 'mkspar2', 'eHistTopx3', 'Escy', 'eHistTopx2', 'injrate', 'mkspar1', 'iHistSEx2', 'irel', 'solw02', 'Na', 'mu', 'equipotm', 'histlimits', 'mksmainionq', 'QnPot', 'version', 'eHistSEx1', 'mksmainionm', 'Escz', 'mksB', 'Nc', 'zg', 'eHistSEx2', 'iHistSEx1', 'iHistTopx3', 'solns02', 'fvperparraycount', 'itertime', 'objects', 'ksi', 'impactdiag01', 'mksTe', 'emmitdiag01', 'objectspowerfluxi', 'deltah', 'eHistTopx1', 'yg', 'objectspowerfluxe', 'mksn0', 'floatconstant', 'mkspar3', 'orientation', 'timehistory

In [144]:
fig, ax = plt.subplots()

ax.plot(splopter.tdata.t_dict['ProbePot'])

for i in range(4):
    line_pos = i * len(splopter.tdata.t_dict['ProbePot']) / 4
    ax.axvline(x=line_pos, **c.AX_LINE_DEFAULTS)

In [140]:
# splopter.plot_2d_variable(b_arrow_loc=(100, 100))
fig, ax = plt.subplots()

# print(splopter.tdata.pot.shape)
# ax.plot(splopter.tdata.pot[:,200])
im = ax.pcolormesh(splopter.tdata.objectsenum)
fig.colorbar(im)

In [145]:
fig, ax = plt.subplots(figsize=[10, 5])

# print(splopter.tdata.pot.shape)
# ax.plot(splopter.tdata.pot[:,200])
im = ax.pcolormesh(splopter.tdata.pot)

ax.set_xlabel(r'y [$\lambda_D$]')
ax.set_ylabel(r'z [$\lambda_D$]')
ax.set_title('Potential')
fig.tight_layout()
ax.axis('scaled')
fig.colorbar(im)

## Other plots

In [147]:
probe_indices = [2,]

I_e = 0.0
I_i = 0.0

tdata = splopter.tdata
for index in probe_indices:
    I_e += np.squeeze(tdata.objectscurrente)[index]
    I_i += np.squeeze(tdata.objectscurrenti)[index]
V = np.squeeze(tdata.diagnostics[c.DIAG_PROBE_POT])

In [148]:
fig, ax = plt.subplots(2, sharex=True)
I_tot = (I_e + I_i)
ax[0].plot(I_tot[::100])

ax[1].plot(I_e[::100], label='e')
ax[1].plot(I_i[::100], label='i')
ax[1].legend()

In [149]:
half_len = int(len(I_tot) / 2)

In [150]:
fig, ax = plt.subplots(2, sharex=True)

ax[0].plot(I_tot[half_len:])

ax[1].plot(I_e[half_len:], label='e')
ax[1].plot(I_i[half_len:], label='i')
ax[1].legend()

/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:749: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:1705: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [151]:
theta = float(spl_path.name.split('_')[2])
pot = float(spl_path.name.split('_')[5])

In [152]:
I_tot_avg = np.mean(I_tot[half_len:])
I_tot_std = np.std(I_tot[half_len:]) / np.sqrt(half_len)

print(I_tot_avg, I_tot_std)

32.76956970663181 0.012406835626540574


In [34]:
# ds = xr.Dataset({'voltage': (['time'], iv['V'][:sweep_length]),
#                  'current': (['time'], iv['I'][:sweep_length]),
#                  'shot_time': (['time'], iv['t'][:sweep_length]),
#                  'start_time': iv['t'][0]},
#                 coords={'time': relative_t[:sweep_length], 'direction': 'up',
#                         'probe': probe_designations[i]})

## Attempting to make a sheath-thickenss measuring function

In [93]:

fig, ax = plt.subplots(2)

im = ax[0].pcolormesh(splopter.tdata.t_dict['rho01'])
fig.colorbar(im, ax=ax[0])
im = ax[1].pcolormesh(splopter.tdata.t_dict['rho02'])
fig.colorbar(im, ax=ax[1])

# x_slice = 200
# for x_slice in np.arange(128,429,10):
#     ax[0].axvline(x=x_slice, **c.AX_LINE_DEFAULTS)
#     ax[1].plot(splopter.tdata.rho[94:,x_slice])


In [94]:
y_slice = slice(94,220)

In [134]:
# x_slices = [0, 280]
# samples = [60, 150]
x_slices = [30, 280]
samples = [30, 150]
n_axes = len(x_slices)

fig, ax = plt.subplots(1, n_axes, figsize=[10, 6])
fig_2d, ax_2d = plt.subplots()
# rho = splopter.tdata.t_dict['rho']
# pot = splopter.tdata.t_dict['Pot']

for i, (x_slc, sample) in enumerate(zip(x_slices, samples)):    
    print(x_slc, sample, x_slc - sample)
    if x_slc - sample < 0:
        roll_offs = np.abs(x_slc-sample)
        rho = np.roll(splopter.tdata.t_dict['rho'], roll_offs)
        pot = np.roll(splopter.tdata.t_dict['Pot'], roll_offs)
  
        x_slc = x_slc + roll_offs
    else:
        rho = splopter.tdata.t_dict['rho']
        pot = splopter.tdata.t_dict['Pot']
    x_slice = slice(x_slc-sample, x_slc+sample)
    print(x_slice)
    ax[0].plot(np.arange(rho[y_slice].shape[0]), np.mean(rho[y_slice, x_slice], axis=1),)
#     ax[i,0].set_xlim(93, 220)
    
    ax[1].plot(np.arange(pot[y_slice].shape[0]), np.mean(pot[y_slice, x_slice], axis=1),)
#     ax[i,1].set_xlim(93, 220)
    
    ax_2d.axvline(x=x_slc, **c.AX_LINE_DEFAULTS)
    ax_2d.axvspan(x_slc + sample, x_slc - sample, color='grey', alpha=0.5)
    
ax_2d.pcolormesh(splopter.tdata.t_dict['Pot'])

fig.tight_layout()

30 30 0
slice(0, 60, None)
280 150 130
slice(130, 430, None)


In [135]:
import more_itertools as mit

fig, ax = plt.subplots()
rho_probe = rho[y_slice, x_slice]
rho_profile = np.mean(rho_probe, axis=1)
rho_qn_profile = np.mean(rho_probe[50:], axis=1)

threshold = 2

ax.plot(rho_profile, color='silver')
ax.plot(np.arange(50, len(rho_probe)), rho_qn_profile)

ax.axhline(y=np.median(rho_qn_profile), **c.AX_LINE_DEFAULTS, label='median')
ax.axhline(y=np.mean(rho_qn_profile), **c.AX_LINE_DEFAULTS, label='mean')
ax.axhline(y=threshold * np.std(rho_qn_profile), **c.AX_LINE_DEFAULTS)

sheath_reg = np.where(np.abs(rho_profile) >= threshold * np.std(rho_qn_profile)) 
ax.plot(np.arange(len(rho_profile))[sheath_reg], rho_profile[sheath_reg], 'x', color='red', alpha=0.5)

ax.axvline(x=np.max(sheath_reg), color='red')
ax.axvline(x=[list(thing)[-1] for thing in (mit.consecutive_groups(sheath_reg[0]))][0], color='blue')

ax.set_ylabel(r'Charge density ($\rho$)')
ax.set_xlabel(r'Distance from probe [$\lambda_D$]')
# ax.pcolormesh(rho[y_slice, x_slice])
print(np.max(sheath_reg))

114


In [136]:
def find_sheath_edge(rho_profile, qn_index=50, threshold=2.5, mode='consec'):
    rho_qn_profile = rho_profile[qn_index:]
    sheath_reg = np.where(np.abs(rho_profile) >= threshold * np.std(rho_qn_profile)) 
    if mode == 'consec':
        return [list(thing)[-1] for thing in (mit.consecutive_groups(sheath_reg[0]))][0]
    elif mode == 'max':
        return np.max(sheath_reg)
    else:
        raise ValueError('You numpty')


In [137]:
fig, ax = plt.subplots(2)

x_slice_probe = slice(250,310)
x_slice_wall = slice(0,60)
rho_probe = splopter.tdata.t_dict['rho'][y_slice, x_slice_probe]
rho_wall = splopter.tdata.t_dict['rho'][y_slice, x_slice_wall]
rho_profile = np.abs(np.mean(rho_probe, axis=1))
rho_qn_profile = rho_profile[50:]

ax[0].plot(rho_profile, color='silver')
ax[0].plot(np.arange(50, len(rho_probe)), rho_qn_profile)

thresholds = np.arange(1, 5.0, 0.5)
for threshold in thresholds:
    ax[0].axhline(y=threshold * np.std(rho_qn_profile), **c.AX_LINE_DEFAULTS)

    
    sheath_edge = find_sheath_edge(rho_profile, threshold=threshold)
    ax[0].axvline(x=sheath_edge, color='tab:blue', lw=0.75, alpha=0.5)
    ax[1].plot(threshold, sheath_edge, 'x', color='tab:blue')
    
    sheath_edge = find_sheath_edge(rho_profile, threshold=threshold, mode='max')
    ax[0].axvline(x=sheath_edge, color='tab:orange', lw=0.75, alpha=0.5)
    ax[1].plot(threshold, sheath_edge, 'o', mfc='none', color='tab:orange')
    
    

In [102]:
# look at change of profile over the length of the probe
def plot_sheath_edge(rho, pot, ax=None, slice_width=25, h_probe=93, probe_x_extent=(130, 431), 
                     yslice_2d_plot=slice(85,220), threshold=2.5):
#     slice_width = 25
#     h_probe = 93
#     yslice_2d_plot = slice(0,220)
#     x_slices = np.arange(130 + slice_width, 431 - slice_width, 2*slice_width)

    x_slices = np.arange(probe_x_extent[0] + slice_width, 
                         probe_x_extent[1] - slice_width, 
                         2 * slice_width)
    
    if ax is None:
        fig, ax = plt.subplots(3, 2, figsize=[8, 10], sharex=True, )
    else:
        fig = ax.figure

#     rho = splopter.tdata.t_dict['rho']
#     pot = splopter.tdata.t_dict['Pot']

    for i, x_slc in enumerate(x_slices): 
        x_slice = slice(x_slc - slice_width, x_slc + slice_width)

        ax[1,0].plot(np.arange(rho[y_slice].shape[0]) + h_probe, np.mean(rho[y_slice, x_slice], axis=1),
                     color=cb_palette[i])

        ax[1,1].plot(np.arange(pot[y_slice].shape[0]) + h_probe, np.mean(pot[y_slice, x_slice], axis=1),
                     color=cb_palette[i])

        ax[2,1].plot(np.arange(pot[y_slice].shape[0]) + h_probe, 
                     -np.gradient(np.gradient(np.mean(pot[y_slice, x_slice], axis=1))),
                     color=cb_palette[i])

        rho_profile = np.abs(np.mean(rho[y_slice, x_slice], axis=1))
        poisson_profile = np.abs(-np.gradient(np.gradient(np.mean(pot[y_slice, x_slice], axis=1))))
        sheath_edge_rho = find_sheath_edge(rho_profile, threshold=threshold)
        sheath_edge_pois = find_sheath_edge(poisson_profile, threshold=threshold)

        for j in range(2):
            ax[0,j].axhline(y=x_slc, lw=1.0, ls='-', color=cb_palette[i])

            ax[0,j].vlines(h_probe + sheath_edge_rho, x_slice.start, x_slice.stop, 
                           lw=1.2, ls='--', color=cb_palette[i])
            ax[1,j].axvline(x=sheath_edge_rho + h_probe, color=cb_palette[i], ls='--')
            
            ax[0,j].vlines(h_probe + sheath_edge_pois, x_slice.start, x_slice.stop, 
                           lw=1.2, ls=':', color=cb_palette[i])
            ax[2,j].axvline(x=sheath_edge_pois + h_probe, color=cb_palette[i], ls=':')

#         ax[0,0].axhspan(x_slc + slice_width, x_slc - slice_width, alpha=0.5,
#                         color=cb_palette[i])


    ax[0,0].pcolormesh(np.transpose(rho[:, :]), cmap='cividis')
    ax[0,1].pcolormesh(np.transpose(pot[:, :]), cmap='cividis')
    
    ax[0,0].set_xlim(yslice_2d_plot.start, yslice_2d_plot.stop)
    
    ax[0,0].set_title(r'$\rho$')
    ax[0,1].set_title(r'$\phi$')

    fig.tight_layout()
    
def get_sheath_edges(rho, pot, slice_width=25, probe_x_extent=(130, 431), threshold=2.5):
    x_slices = np.arange(probe_x_extent[0] + slice_width, 
                         probe_x_extent[1] - slice_width, 
                         2 * slice_width)
    se_rhos = []
    se_poiss = []
    for i, x_slc in enumerate(x_slices): 
        x_slice = slice(x_slc - slice_width, x_slc + slice_width)

        rho_profile = np.abs(np.mean(rho[y_slice, x_slice], axis=1))
        poisson_profile = np.abs(-np.gradient(np.gradient(np.mean(pot[y_slice, x_slice], axis=1))))
        
        se_rhos.append(find_sheath_edge(rho_profile, threshold=threshold))
        se_poiss.append(find_sheath_edge(poisson_profile, threshold=threshold))
    return x_slices, se_rhos, se_poiss

    
rho = splopter.tdata.t_dict['rho']
pot = splopter.tdata.t_dict['Pot']
plot_sheath_edge(rho, pot)

Looks like it's most consistent to use threshold=2 and mode='consec'

# Combine it all together!

In [160]:
labels = ['theta', 'voltage', 
          'current', 'd_current', 'derr_current', 
          'current_e', 'd_current_e', 'derr_current_e', 
          'current_i', 'd_current_i', 'derr_current_i']
pw_df = pd.DataFrame(columns=labels)

In [161]:
probe_indices = [2,]

for i, scan in enumerate(scans):
    for j, run in enumerate(all_run_dirs[scan]):
        spl_path = spice_dir / run
        bups = list(spl_path.glob('backup*'))
        bups.sort()
        final_state_path = spl_path / bups[-1].name
        splopter =  spl.Splopter(final_state_path, reduce=desired_variables, ignore_tzero_fl=True, 
                                 check_voltage_error_fl=False)
        splopter.prepare(find_se_temp_fl=False, homogenise_fl=False)
        
        I_e = 0.0
        I_i = 0.0
        for index in probe_indices:
            I_e += np.squeeze(splopter.tdata.objectscurrente)[index]
            I_i += np.squeeze(splopter.tdata.objectscurrenti)[index]
            print(np.isnan(I_e))
        I_tot = I_e + I_i
        
        pw_df = pw_df.append({
            'theta': float(spl_path.name.split('_')[2]),
            'voltage': float(spl_path.name.split('_')[5]),
            'current': np.mean(I_tot[half_len:]),
            'd_current': np.std(I_tot[half_len:]),
            'derr_current': np.std(I_tot[half_len:]) / np.sqrt(half_len),
            'current_i': np.mean(I_i[half_len:]),
            'd_current_i': np.std(I_i[half_len:]),
            'derr_current_i': np.std(I_i[half_len:]) / np.sqrt(half_len),
            'current_e': np.mean(I_e[half_len:]),
            'd_current_e': np.std(I_e[half_len:]),
            'derr_current_e': np.std(I_e[half_len:]) / np.sqrt(half_len)
        }, ignore_index=True)
        
        
pw_df       

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush/alpha_yz_-3.0__pot_-1.0/backup_20201212-1437) doesn't seem to be valid.
Continuing anyway.
[False False False ... False False False]
Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush/alpha_yz_-3.0__pot_-10.0/backup_20201212-1322) doesn't seem to be valid.
Continuing anyway.
[False False False ... False False False]
Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush/alpha_yz_-3.0__pot_-12.0/backup_20201212-1127) doesn't seem to be valid.
Continuing anyway.
[False False False ... False False False]
Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi

,theta,voltage,current,d_current,derr_current,current_e,d_current_e,derr_current_e,current_i,d_current_i,derr_current_i
0,-3.0,-1.0,-46.873286,11.884515,0.024836,-53.290174,12.461397,0.026041,6.416888,2.799292,0.005850
1,-3.0,-10.0,9.794869,3.541105,0.007400,-0.021500,0.146853,0.000307,9.816369,3.540355,0.007398
2,-3.0,-12.0,9.941793,3.556028,0.007431,-0.002822,0.053084,0.000111,9.944615,3.555913,0.007431
3,-3.0,-18.0,10.230288,3.591697,0.007506,-0.000011,0.003295,0.000007,10.230299,3.591703,0.007506
4,-3.0,-2.0,-32.896150,9.787258,0.020453,-40.432851,10.349384,0.021628,7.536701,3.061401,0.006398
...,...,...,...,...,...,...,...,...,...,...,...
93,-10.0,-25.0,34.060317,6.023134,0.012587,-0.000004,0.002090,0.000004,34.060322,6.023135,0.012587
94,-12.0,-18.0,36.927916,6.141905,0.012835,-0.000039,0.006253,0.000013,36.927955,6.141891,0.012835
95,-12.0,-25.0,38.694512,6.209786,0.012977,-0.000013,0.003610,0.000008,38.694525,6.209792,0.012977
96,-8.0,-18.0,27.435118,5.355500,0.011192,-0.000029,0.005405,0.000011,27.435147,5.355500,0.011192


In [162]:
pw_df.loc[pw_df.theta == -8.0].set_index('voltage').to_xarray().drop('theta').expand_dims({'theta':[-8.0]})

<xarray.Dataset>
Dimensions:         (theta: 1, voltage: 14)
Coordinates:
  * theta           (theta) float64 -8.0
  * voltage         (voltage) float64 -1.0 -10.0 -12.0 -2.0 ... 0.0 -18.0 -25.0
Data variables:
    current         (theta, voltage) float64 -135.6 26.11 26.54 ... 27.44 28.13
    d_current       (theta, voltage) float64 13.93 5.256 5.284 ... 5.356 5.408
    derr_current    (theta, voltage) float64 0.02912 0.01098 ... 0.01119 0.0113
    current_e       (theta, voltage) float64 -150.3 -0.05049 ... -2.921e-06
    d_current_e     (theta, voltage) float64 13.63 0.2243 ... 0.005405 0.001709
    derr_current_e  (theta, voltage) float64 0.02848 0.0004688 ... 3.572e-06
    current_i       (theta, voltage) float64 14.75 26.17 26.55 ... 27.44 28.13
    d_current_i     (theta, voltage) float64 3.924 5.252 5.284 ... 5.355 5.408
    derr_current_i  (theta, voltage) float64 0.0082 0.01098 ... 0.01119 0.0113

In [166]:
combo_ds = xr.concat([pw_df.loc[pw_df.theta == theta].set_index('voltage').to_xarray().drop('theta').expand_dims({'theta':[theta]}) 
 for theta in [-3.0,-5.0,-8.0,-10.0,-12.0,-15.0,-20.0]], dim='theta')

In [13]:
combo_ds['theta_rads'] = np.radians(-combo_ds['theta'])
# combo_ds['theta_rads'] = -combo_ds['theta']
combo_ds = combo_ds.expand_dims({'probe': ['flat_flush']})
combo_ds['theta_p_rads'] = combo_ds['probe'] 
combo_ds['recession'] = combo_ds['probe'] 
# combo_ds = combo_ds.sel(probe='flat_flush')

ValueError: Dimension probe already exists.

In [177]:
combo_ds

<xarray.Dataset>
Dimensions:         (probe: 1, theta: 7, voltage: 14, y: 557, z: 233)
Coordinates:
  * probe           (probe) <U10 'flat_flush'
  * z               (z) int64 0 1 2 3 4 5 6 7 ... 226 227 228 229 230 231 232
  * voltage         (voltage) float64 -25.0 -18.0 -12.0 -10.0 ... -2.0 -1.0 0.0
  * y               (y) int64 0 1 2 3 4 5 6 7 ... 550 551 552 553 554 555 556
  * theta           (theta) float64 3.0 5.0 15.0 20.0 12.0 10.0 8.0
Data variables:
    current         (probe, theta, voltage) float64 10.47 10.23 ... -184.3
    d_current       (probe, theta, voltage) float64 3.618 3.592 ... 13.93 16.11
    derr_current    (probe, theta, voltage) float64 0.007561 ... 0.03366
    current_i       (probe, theta, voltage) float64 10.47 10.23 ... 14.75 10.22
    d_current_i     (probe, theta, voltage) float64 3.618 3.592 ... 3.924 3.235
    derr_current_i  (probe, theta, voltage) float64 0.007561 ... 0.006761
    current_e       (probe, theta, voltage) float64 -3.1e-06 ... -194.6
    d_current_e     (probe, theta, voltage) float64 0.001761 0.003295 ... 15.97
    derr_current_e  (probe, theta, voltage) float64 3.68e-06 ... 0.03337
    rho             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 111.5
    pot             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 0.0 0.0
    delta           (probe, theta, voltage) int64 32 26 19 16 12 ... 5 2 3 1 0
    delta_w         (probe, theta, voltage) int64 32 26 19 16 12 ... 5 2 3 1 0
    theta_rads      (probe, theta) float64 -0.05236 -0.08727 ... -0.1745 -0.1396
    theta_p_rads    (probe) <U10 'flat_flush'
    recession       (probe) <U10 'flat_flush'

In [178]:
combo_ds.to_netcdf('piecewise_sheath_exp_ivs.nc')

In [14]:
combo_ds = xr.load_dataset('piecewise_sheath_exp_ivs.nc')
combo_ds

<xarray.Dataset>
Dimensions:         (probe: 1, theta: 7, voltage: 14, y: 557, z: 233)
Coordinates:
  * probe           (probe) object 'flat_flush'
  * z               (z) int64 0 1 2 3 4 5 6 7 ... 226 227 228 229 230 231 232
  * voltage         (voltage) float64 -25.0 -18.0 -12.0 -10.0 ... -2.0 -1.0 0.0
  * y               (y) int64 0 1 2 3 4 5 6 7 ... 550 551 552 553 554 555 556
  * theta           (theta) float64 3.0 5.0 15.0 20.0 12.0 10.0 8.0
Data variables:
    current         (probe, theta, voltage) float64 10.47 10.23 ... -184.3
    d_current       (probe, theta, voltage) float64 3.618 3.592 ... 13.93 16.11
    derr_current    (probe, theta, voltage) float64 0.007561 ... 0.03366
    current_i       (probe, theta, voltage) float64 10.47 10.23 ... 14.75 10.22
    d_current_i     (probe, theta, voltage) float64 3.618 3.592 ... 3.924 3.235
    derr_current_i  (probe, theta, voltage) float64 0.007561 ... 0.006761
    current_e       (probe, theta, voltage) float64 -3.1e-06 ... -194.6
    d_current_e     (probe, theta, voltage) float64 0.001761 0.003295 ... 15.97
    derr_current_e  (probe, theta, voltage) float64 3.68e-06 ... 0.03337
    rho             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 111.5
    pot             (probe, theta, voltage, z, y) float64 0.0 0.0 ... 0.0 0.0
    delta           (probe, theta, voltage) int64 32 26 19 16 12 ... 5 2 3 1 0
    delta_w         (probe, theta, voltage) int64 32 26 19 16 12 ... 5 2 3 1 0
    theta_rads      (probe, theta) float64 -0.05236 -0.08727 ... -0.1745 -0.1396
    theta_p_rads    (probe) object 'flat_flush'
    recession       (probe) object 'flat_flush'

In [16]:
combo_ds['recession'] = 0.0

In [18]:
# swept_ds = xr.load_dataset('new_sheath_exp_ivs.nc')
old_swept_ds = xr.load_dataset('sheath_exp_fwp_ivs.nc')
swept_ds = xr.load_dataset('sheath_exp_hg_ivs.nc')
swept_ds

<xarray.Dataset>
Dimensions:               (theta: 15, voltage: 1201)
Coordinates:
  * voltage               (voltage) float64 -29.98 -29.95 ... 1.665e-15 0.025
  * theta                 (theta) float64 2.0 3.0 4.0 5.0 ... 15.0 20.0 30.0
    theta_p               float64 0.0
    theta_p_rads          float64 0.0
    theta_rads            (theta) float64 0.03491 0.05236 ... 0.3491 0.5236
Data variables:
    str_iv_chi2           (theta) float64 723.8 802.2 ... 1.07e+03 1.067e+03
    str_iv_reduced_chi2   (theta) float64 0.6514 0.7195 0.738 ... 0.9598 0.9566
    str_iv_I_sat          (theta) float64 -14.44 -21.82 -29.39 ... -108.4 -150.9
    str_iv_d_I_sat        (theta) float64 0.01673 0.02617 ... 0.1702 0.2316
    str_iv_a              (theta) float64 0.005932 0.005801 ... 0.01262 0.008279
    str_iv_d_a            (theta) float64 0.0001485 0.0001528 ... 0.0001997
    str_iv_T_e            (theta) float64 1.113 1.099 1.089 ... 1.082 1.076
    str_iv_d_T_e          (theta) float64 0.005743 0.005632 ... 0.005819 0.00541
    str_iv_V_f            (theta) float64 -2.368 -2.338 -2.332 ... -2.666 -2.736
    str_iv_d_V_f          (theta) float64 0.003084 0.002892 ... 0.002625
    str_iv_run_dir        (theta) object 'marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-2.0' ... 'marconi/spice2/sheath_exp_hg/flat_flush_as/alpha_yz_-30.0'
    str_iv_voltage_min    (theta) float64 -29.98 -29.98 -29.98 ... -29.98 -29.98
    str_iv_voltage_max    (theta) float64 -2.125 -2.025 -2.025 ... -2.025 -2.025
    norm_iv_chi2          (theta) float64 1.106e+03 1.085e+03 ... 1.176e+03
    norm_iv_reduced_chi2  (theta) float64 1.003 0.9827 0.9814 ... 1.051 1.081
    norm_iv_I_sat         (theta) float64 -14.2 -21.51 -29.0 ... -107.3 -149.3
    norm_iv_d_I_sat       (theta) float64 0.01626 0.02425 ... 0.1448 0.2018
    norm_iv_a             (theta) float64 0.007396 0.007055 ... 0.01315 0.009172
    norm_iv_d_a           (theta) float64 0.0001542 0.0001512 ... 0.0001874
    norm_iv_voltage_min   (theta) float64 -27.61 -27.64 -27.65 ... -27.29 -27.25
    norm_iv_voltage_max   (theta) float64 -0.01163 -0.0166 ... -0.01911 -0.0208
    ion_chi2              (theta) float64 528.0 593.0 568.0 ... 820.2 776.8
    ion_reduced_chi2      (theta) float64 0.6118 0.6864 0.6566 ... 0.9649 0.916
    ion_I_sat             (theta) float64 14.46 21.91 29.43 ... 108.5 150.8
    ion_d_I_sat           (theta) float64 0.01913 0.03042 ... 0.2098 0.2797
    ion_a                 (theta) float64 0.005252 0.004938 ... 0.01178 0.007918
    ion_d_a               (theta) float64 0.0001623 0.0001699 ... 0.0002357
    ion_voltage_min       (theta) float64 3.839 3.842 3.834 ... 3.843 3.844
    ion_voltage_max       (theta) float64 12.05 12.06 12.06 ... 11.94 11.93
    current               (theta, voltage) float64 -15.88 -15.98 ... -0.0 nan
    current_e             (theta, voltage) float64 0.0 0.0 0.0 ... 0.0 nan
    current_i             (theta, voltage) float64 15.88 15.98 16.02 ... 0.0 nan
    d_current             (theta, voltage) float64 0.1944 0.2147 ... 0.0 nan
    d_current_e           (theta, voltage) float64 0.0 0.0 0.0 ... 7.594 0.0 nan
    d_current_i           (theta, voltage) float64 0.1944 0.2147 ... 0.0 nan
    time                  (theta, voltage) float64 0.0 0.06436 ... 5.294 nan

## Attempt to make a dataset straight up with 2d information included

In [171]:
print(splopter.tdata.pot.shape)
splopter.tdata.ny

(233, 557)


array([[557]], dtype=int32)

In [173]:
probe_indices = [2,]
ds_list_thetas = {}
voltages = []
thetas = set()

for i, scan in enumerate(scans):
    for j, run in enumerate(all_run_dirs[scan]):
        spl_path = spice_dir / run
        bups = list(spl_path.glob('backup*'))
        bups.sort()
        final_state_path = spl_path / bups[-1].name
        splopter =  spl.Splopter(final_state_path, reduce=desired_variables, ignore_tzero_fl=True, 
                                 check_voltage_error_fl=False)
        splopter.prepare(find_se_temp_fl=False, homogenise_fl=False)
        
        # Get ion and electron currents into probe objects
        I_e = 0.0
        I_i = 0.0
        for index in probe_indices:
            I_e += np.squeeze(splopter.tdata.objectscurrente)[index]
            I_i += np.squeeze(splopter.tdata.objectscurrenti)[index]
            print(f'{theta}:{pot} - {np.isnan(I_i)}')
        I_tot = I_e + I_i
        
        # specifically looking at the plasma in front of the probe
        x_slice_p = slice(130, 430)
        x_slice_w = slice(0,60)
        y_slice = slice(94, 220)
        rho_probe = splopter.tdata.rho[y_slice, x_slice_p]
        rho_wall = splopter.tdata.rho[y_slice, x_slice_w]
        
        voltage = float(spl_path.name.split('_')[5])
        theta = -float(spl_path.name.split('_')[2])
        
        ds = xr.Dataset(
            {
                'current': np.mean(I_tot[half_len:]),
                'd_current': np.std(I_tot[half_len:]),
                'derr_current': np.std(I_tot[half_len:]) / np.sqrt(half_len),
                'current_i': np.mean(I_i[half_len:]),
                'd_current_i': np.std(I_i[half_len:]),
                'derr_current_i': np.std(I_i[half_len:]) / np.sqrt(half_len),
                'current_e': np.mean(I_e[half_len:]),
                'd_current_e': np.std(I_e[half_len:]),
                'derr_current_e': np.std(I_e[half_len:]) / np.sqrt(half_len),
                'rho': (['z', 'y'], splopter.tdata.rho),
                'pot': (['z', 'y'], splopter.tdata.pot),
                'delta': find_sheath_edge(np.abs(np.mean(rho_probe, axis=1)), threshold=2.0),
                'delta_w': find_sheath_edge(np.abs(np.mean(rho_wall, axis=1)), threshold=2.0)
            },
            coords={
                'voltage': voltage,
                'theta': theta,
                'z': np.arange(splopter.tdata.nz),
                'y': np.arange(splopter.tdata.ny)
            }
        )
        if theta in ds_list_thetas:
            ds_list_thetas[theta].append(ds)
        else:
            ds_list_thetas[theta] = [ds]
#         if theta in voltages_thetas:
#             voltages_thetas[theta].append(ds)
#         else:
#             voltages_thetas[theta] = [ds]
#         ds_list.append(ds)
        voltages.append(voltage)
        thetas.add(theta)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush/alpha_yz_-3.0__pot_-1.0/backup_20201212-1437) doesn't seem to be valid.
Continuing anyway.
3.0:-18.0 - [False False False ... False False False]
Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush/alpha_yz_-3.0__pot_-10.0/backup_20201212-1322) doesn't seem to be valid.
Continuing anyway.
3.0:-18.0 - [False False False ... False False False]
Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/piecewise/flat_flush/alpha_yz_-3.0__pot_-12.0/backup_20201212-1127) doesn't seem to be valid.
Continuing anyway.
3.0:-18.0 - [False False False ... False False False]
Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jle

In [174]:
for k, v in ds_list_thetas.items():
    print(k)
    for ds in v:
        print('\t', ds.theta.values, ds.voltage.values)

3.0
	 3.0 -1.0
	 3.0 -10.0
	 3.0 -12.0
	 3.0 -18.0
	 3.0 -2.0
	 3.0 -25.0
	 3.0 -3.0
	 3.0 -4.0
	 3.0 -5.0
	 3.0 -6.0
	 3.0 -7.0
	 3.0 -8.0
	 3.0 -9.0
	 3.0 0.0
5.0
	 5.0 -1.0
	 5.0 -10.0
	 5.0 -12.0
	 5.0 -18.0
	 5.0 -2.0
	 5.0 -25.0
	 5.0 -3.0
	 5.0 -4.0
	 5.0 -5.0
	 5.0 -6.0
	 5.0 -7.0
	 5.0 -8.0
	 5.0 -9.0
	 5.0 0.0
15.0
	 15.0 -1.0
	 15.0 -10.0
	 15.0 -12.0
	 15.0 -18.0
	 15.0 -2.0
	 15.0 -25.0
	 15.0 -3.0
	 15.0 -4.0
	 15.0 -5.0
	 15.0 -6.0
	 15.0 -7.0
	 15.0 -8.0
	 15.0 -9.0
	 15.0 0.0
20.0
	 20.0 -1.0
	 20.0 -10.0
	 20.0 -12.0
	 20.0 -18.0
	 20.0 -2.0
	 20.0 -25.0
	 20.0 -3.0
	 20.0 -4.0
	 20.0 -5.0
	 20.0 -6.0
	 20.0 -7.0
	 20.0 -8.0
	 20.0 -9.0
	 20.0 0.0
12.0
	 12.0 -12.0
	 12.0 -1.0
	 12.0 -10.0
	 12.0 -2.0
	 12.0 -3.0
	 12.0 -4.0
	 12.0 -5.0
	 12.0 -6.0
	 12.0 -7.0
	 12.0 -8.0
	 12.0 -9.0
	 12.0 0.0
	 12.0 -18.0
	 12.0 -25.0
10.0
	 10.0 -1.0
	 10.0 -10.0
	 10.0 -12.0
	 10.0 -2.0
	 10.0 -3.0
	 10.0 -4.0
	 10.0 -5.0
	 10.0 -6.0
	 10.0 -7.0
	 10.0 -8.0
	 10.0 -9.0
	 10.0 0.0


In [175]:
# voltage_da = xr.DataArray(voltages, dims=['voltage'], name='voltage')
# theta_da = xr.DataArray(thetas, dims=['theta'], name='theta')
voltage_dss = []
for theta, ds_list in ds_list_thetas.items():
    print(theta)
    voltage_dss.append(xr.concat(ds_list, dim='voltage'))
combo_ds = xr.concat(voltage_dss, dim='theta')
combo_ds

3.0
5.0
15.0
20.0
12.0
10.0
8.0


<xarray.Dataset>
Dimensions:         (theta: 7, voltage: 14, y: 557, z: 233)
Coordinates:
  * z               (z) int64 0 1 2 3 4 5 6 7 ... 226 227 228 229 230 231 232
  * voltage         (voltage) float64 -25.0 -18.0 -12.0 -10.0 ... -2.0 -1.0 0.0
  * y               (y) int64 0 1 2 3 4 5 6 7 ... 550 551 552 553 554 555 556
  * theta           (theta) float64 3.0 5.0 15.0 20.0 12.0 10.0 8.0
Data variables:
    current         (theta, voltage) float64 10.47 10.23 9.942 ... -135.6 -184.3
    d_current       (theta, voltage) float64 3.618 3.592 3.556 ... 13.93 16.11
    derr_current    (theta, voltage) float64 0.007561 0.007506 ... 0.03366
    current_i       (theta, voltage) float64 10.47 10.23 9.945 ... 14.75 10.22
    d_current_i     (theta, voltage) float64 3.618 3.592 3.556 ... 3.924 3.235
    derr_current_i  (theta, voltage) float64 0.007561 0.007506 ... 0.006761
    current_e       (theta, voltage) float64 -3.1e-06 -1.085e-05 ... -194.6
    d_current_e     (theta, voltage) float64 0.001761 0.003295 ... 13.63 15.97
    derr_current_e  (theta, voltage) float64 3.68e-06 6.885e-06 ... 0.03337
    rho             (theta, voltage, z, y) float64 0.0 0.0 0.0 ... 127.2 111.5
    pot             (theta, voltage, z, y) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    delta           (theta, voltage) int64 32 26 19 16 12 11 13 ... 9 5 2 3 1 0
    delta_w         (theta, voltage) int64 32 26 19 16 12 11 13 ... 9 5 2 3 1 0

In [176]:
combo_ds['theta_rads'] = np.radians(-combo_ds['theta'])
# combo_ds['theta_rads'] = -combo_ds['theta']
combo_ds = combo_ds.expand_dims({'probe': ['flat_flush']})
combo_ds['theta_p_rads'] = combo_ds['probe'] 
combo_ds['recession'] = combo_ds['probe'] 
# combo_ds = combo_ds.sel(probe='flat_flush')

In [183]:
combo_ds = combo_ds.sortby('theta')

## Start looking at data

In [17]:
fig, axes = plt.subplots(1, 3, sharex=True, sharey=True, figsize=[10,6])

for i, theta in enumerate([8.0, 10.0, 12.0]):
    ax = axes[i]
    fixed_plot_ds = combo_ds.sel(voltage=slice(-25.0,0), theta=theta)
    sweep_plot_ds = swept_ds.sel(theta_p=0.0, recession=0.0, theta=theta, voltage=slice(-10,0))

    (-fixed_plot_ds['current']).plot.line(x='voltage', ax=ax, label='piecewise', linestyle='none', marker='x')
    sweep_plot_ds['current'].plot.line(x='voltage', ax=ax, label=r'swept - $V_{wall}$ floating')

    ax.axhline(y=0, **c.AX_LINE_DEFAULTS)
    ax.set_title(r'$\theta = {}$'.format(theta))
    ax.legend()

fig.tight_layout()

ValueError: dimensions or multi-index levels ['theta_p', 'recession'] do not exist

In [193]:
fig, axes = plt.subplots(2, 4, sharex=True, sharey=True, figsize=[10,6])

for i, theta in enumerate(combo_ds.theta.values):
    ax = axes[i//4][i%4]
    fixed_plot_ds = combo_ds.sel(voltage=slice(-25.0,-4.0), theta=theta)

    if theta in swept_ds.theta:
        sweep_plot_ds = swept_ds.sel(probe='flat_flush', theta=theta, voltage=slice(-25.0,-4.0))
        sweep_plot_ds['current'].plot.line(x='voltage', ax=ax, label=r'swept - $V_{wall}$ floating')


    (-fixed_plot_ds['current']).plot.line(x='voltage', ax=ax, label='piecewise', linestyle='none', marker='x')
    ax.axhline(y=0, **c.AX_LINE_DEFAULTS)
    ax.set_title(r'$\theta = {}$'.format(theta))
#     ax.set_ylim(None, 0)
    ax.legend()

fig.tight_layout()

In [194]:
fig, ax = plt.subplots(1, 2, sharey=True, figsize=[8,6])

se_fitter = fts.IonCurrentSEFitter()
thetas = []
seps = []
d_seps = []
for theta in combo_ds.theta.values:
    fixed_plot_ds = combo_ds.sel(theta=theta, voltage=slice(-25,-10))

    cl_voltage = np.float_power(np.abs(fixed_plot_ds.voltage.values + 4), 0.75)
    
#     ax[0].plot(fixed_plot_ds.pot.values, fixed_plot_ds.I_e.values)
    ax[0].plot(fixed_plot_ds.voltage.values, fixed_plot_ds.current_i.values, label=r'$\theta=${}'.format(theta))
    ax[0].set_ylabel(r'$I_i$')
    ax[0].set_xlabel(r'$V_p$')
    ax[0].legend()
    
    ax[1].plot(cl_voltage, fixed_plot_ds.current_i.values)
    fit_data = se_fitter.fit(cl_voltage, fixed_plot_ds.current_i.values, sigma=fixed_plot_ds.derr_current_i.values)
    ax[1].plot(*fit_data.get_fit_plottables(), label='a={:.3g}'.format(fit_data.get_sheath_exp()))
    ax[1].legend()
    ax[1].set_xlabel(r'$|V_p - V_f|^{\frac{3}{4}}$')
    
    thetas.append(theta)
    seps.append(fit_data.get_sheath_exp())
    d_seps.append(fit_data.get_sheath_exp_err())
    
fig.tight_layout()

ValueError: x and y must have same first dimension, but have shapes (4,) and (1, 4)

In [46]:
print(thetas, seps, 1/np.tan(-np.radians(thetas)))

[-8.0, -10.0, -12.0] [0.013246865664096224, 0.018304079543884168, 0.021236248601817248] [7.11536972 5.67128182 4.70463011]


In [47]:
combo_ds['ion_a'] = (combo_ds['theta'] * 0) + np.array(seps)

In [52]:
fig, ax = plt.subplots()

plot_decomp(combo_ds, sheath_label='ion_a', mode=1, ax=ax, colour='tab:blue')


(<matplotlib.axes._subplots.AxesSubplot at 0x7f0565a5d438>,
 <flopter.core.fitdata.FitData2 at 0x7f0565dd0860>)

## Vary the trimming to test sensitivity

In [54]:
trim_values = np.flip(combo_ds.voltage.values[2:-5])
trim_values

array([ -5.,  -6.,  -7.,  -8.,  -9., -10., -12., -14., -16., -18.])

In [58]:
fig, ax = plt.subplots(1, 2, figsize=[10,6])

# combo_ds.sel(pot=slice(-25,-4))['I_tot'].plot.line(x='pot', hue='theta', ax=ax[0])

for i, lower_trim in enumerate(trim_values):
    se_fitter = fts.IonCurrentSEFitter()
    thetas = []
    seps = []
    d_seps = []
    ax[0].axvline(x=np.float_power(np.abs(lower_trim + 4), 3/4), **c.AX_LINE_DEFAULTS)
    for theta in combo_ds.theta.values:
        fixed_plot_ds = combo_ds.sel(theta=theta, voltage=slice(-25, lower_trim))

        cl_voltage = np.float_power(np.abs(fixed_plot_ds.voltage.values + 4), 0.75)
        
        if i == 0:
            all_plot_ds = combo_ds.sel(theta=theta, voltage=slice(-25,-4))
            ax[0].plot(np.float_power(np.abs(all_plot_ds.voltage.values + 4), 0.75), 
                       all_plot_ds.current_i.values, label=theta)
            ax[0].set_ylabel(r'$I_i$')
            ax[0].set_xlabel(r'$|V|^{\frac{3}{4}}$')
            ax[0].legend()
        
        fit_data = se_fitter.fit(cl_voltage, fixed_plot_ds.current_i.values, sigma=fixed_plot_ds.derr_current_i.values)
        
        thetas.append(theta)
        seps.append(fit_data.get_sheath_exp())
        d_seps.append(fit_data.get_sheath_exp_err())
        
    combo_ds['ion_a'] = (combo_ds['theta'] * 0) + np.array(seps)

    plot_decomp(combo_ds, sheath_label='ion_a', mode=1, ax=ax[1], colour=cb_palette[i], plot_label=lower_trim)
ax[1].legend(bbox_to_anchor=(1.0, 1.05), loc='upper center', borderaxespad=0.)
fig.tight_layout()

## Compare between fixed-wall-potential, piecewise concatenation and floating potential sims
All at $\theta = 10^{\circ}$

In [40]:
fwp_dir_path = list((spice_dir / 'marconi/spice2/sheath_exp_fwp/flat_10').glob('*'))[0]
print(fwp_dir_path)
fwp_backup_path = list(fwp_dir_path.glob('backup*'))[0]


/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_10/alpha_yz_-10.0


In [41]:
splopter =  spl.Splopter(fwp_backup_path, reduce=desired_variables, ignore_tzero_fl=True, version=2.14,
                         store_dataframe_fl=True, )

splopter.prepare(find_se_temp_fl=False, homogenise_fl=True, backup_concat_fl=False)

Spice data directory is not valid, attempting to auto-fix.
Passed Spice directory (/home/jleland/data/external_big/spice/marconi/spice2/sheath_exp_fwp/flat_10/alpha_yz_-10.0/backup_20201117-1742) doesn't seem to be valid.
Continuing anyway.


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/scipy/io/matlab/mio.py:218: MatReadWarning: Duplicate variable name "npartproc" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [42]:
splopter.iv_data.plot()

<ErrorbarContainer object of 3 artists>

In [43]:
new_iv_data = splopter.iv_data.get_below_floating()

In [44]:
fig, ax = plt.subplots()

sweep_plot_ds = swept_ds.sel(theta_p=0.0, recession=0.0, theta=10, voltage=slice(-10,0))
pw_plot_ds = combo_ds.sel(theta=10.0, voltage=slice(-10,0))

splopter.iv_data.plot(ax=ax)
sweep_plot_ds['current'].plot.line(x='voltage', ax=ax, label='')
(-pw_plot_ds['current']).plot.line(x='voltage', ax=ax)

In [45]:
se_fitter = fts.IonCurrentSEFitter()
v_f = splopter.iv_data.get_vf()
sweep_plot_ds = swept_ds.sel(theta_p=0.0, recession=0.0, theta=10, voltage=slice(-10, v_f))
pw_plot_ds = combo_ds.sel(theta=10.0, voltage=slice(-25, v_f))

fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=[12,6])
ax[0].plot(np.float_power(np.abs(new_iv_data['V'] - v_f), 0.75), new_iv_data['I_i'])
ax[0].set_title('Fixed wall potential')

fit_data = se_fitter.fit(np.float_power(np.abs(new_iv_data['V'] - v_f), 0.75), new_iv_data['I_i'])
fit_data.plot(ax=ax[0])

ax[1].plot(np.float_power(np.abs(sweep_plot_ds['voltage'] - v_f), 0.75), sweep_plot_ds['current_i'])
fit_data = se_fitter.fit(np.float_power(np.abs(sweep_plot_ds['voltage'] - v_f), 0.75), sweep_plot_ds['current_i'])
fit_data.plot(ax=ax[1])
ax[1].set_title('Floating wall potential')

ax[0].plot(np.float_power(np.abs(pw_plot_ds['voltage'] - v_f), 0.75), pw_plot_ds['current_i'])
fit_data = se_fitter.fit(np.float_power(np.abs(pw_plot_ds['voltage'] - v_f), 0.75), pw_plot_ds['current_i'])
fit_data.plot(ax=ax[0])
ax[2].set_title('Fixed wall potential - Piecewise')


fig.tight_layout()

## Check the sheath size as a function of angle and voltage

In [1]:

sl_fitter = fts.StraightLineFitter()

fig, ax = plt.subplots()

delta_0s = []

for theta in combo_ds.theta.values:
    pw_plot_ds = combo_ds.sel(theta=theta, voltage=slice(-25, v_f))
    
    cl_voltage = np.float_power(np.abs(pw_plot_ds.voltage.values - v_f), 0.75)
    ax.plot(cl_voltage, pw_plot_ds.delta.values, 'x', label=r'$\theta = {}$'.format(theta))
    
    delta_fit_data = sl_fitter.fit(cl_voltage, pw_plot_ds.delta.values)
    ax.plot(*delta_fit_data.get_fit_plottables(), 
            label=r'$\Delta_0 = {:.3g}, \Delta_1 = {:.3g}$'.format(delta_fit_data.get_param('y_0'), 
                                                                   delta_fit_data.get_param('m')))
    ax.legend()
    
    delta_0s.append(delta_fit_data.get_param('y_0'))

ax.set_ylabel(r'$\Delta$')
ax.set_xlabel(r'$|V|^{3/4}$')

delta_0s = np.array(delta_0s)

NameError: name 'fts' is not defined

In [66]:

sl_fitter = fts.StraightLineFitter()

fig, ax = plt.subplots(figsize=[6,6])

delta_0s = []

for i, theta in enumerate(combo_ds.theta.values):
    pw_plot_ds = combo_ds.sel(theta=theta, voltage=slice(-25, v_f))
    
    cl_voltage = np.float_power(np.abs(pw_plot_ds.voltage.values - v_f), 0.75)
    
    delta_fit_data = sl_fitter.fit(cl_voltage, pw_plot_ds.delta.values)
    
    delta_0 = delta_fit_data.get_param('y_0')
    flattened_delta = (pw_plot_ds.delta.values - delta_0) * np.sqrt(np.sin(np.radians(theta)))
    
    ax.plot(cl_voltage, flattened_delta, 'x', label=r'$\theta = {}$'.format(theta))
    
    delta_fit_data = sl_fitter.fit(cl_voltage, flattened_delta)
    ax.plot(*delta_fit_data.get_fit_plottables(), 
            label=r'$y_0 = {:.3g}$'.format(delta_fit_data.get_param('y_0')))
    ax.legend()


ax.set_ylabel(r'$(\Delta - \Delta_0) \sin^{1/2}{\theta} $')
ax.set_xlabel(r'$|V|^{3/4}$')

delta_0s = np.array(delta_0s)

fig.tight_layout()

In [49]:
fig, ax = plt.subplots()

for voltage in combo_ds.voltage.values:
    ax.plot(np.sqrt(np.sin(np.radians(combo_ds.theta.values))), 
            combo_ds.sel(voltage=voltage).delta.values - delta_0s)


In [2]:
fig, ax = plt.subplots()

plot_ds = combo_ds.sel(voltage=-20.0)

ax.plot(np.sqrt(np.sin(np.radians(plot_ds.theta.values))), plot_ds.delta.values - delta_0s)


NameError: name 'plt' is not defined

In [47]:
n = combo_ds.voltage.size
print(n)
color = plt.cm.viridis(np.linspace(0, 1, n))
color

17


array([[0.267004, 0.004874, 0.329415, 1.      ],
       [0.282327, 0.094955, 0.417331, 1.      ],
       [0.278826, 0.17549 , 0.483397, 1.      ],
       [0.258965, 0.251537, 0.524736, 1.      ],
       [0.229739, 0.322361, 0.545706, 1.      ],
       [0.19943 , 0.387607, 0.554642, 1.      ],
       [0.172719, 0.448791, 0.557885, 1.      ],
       [0.149039, 0.508051, 0.55725 , 1.      ],
       [0.127568, 0.566949, 0.550556, 1.      ],
       [0.120638, 0.625828, 0.533488, 1.      ],
       [0.157851, 0.683765, 0.501686, 1.      ],
       [0.24607 , 0.73891 , 0.452024, 1.      ],
       [0.369214, 0.788888, 0.382914, 1.      ],
       [0.515992, 0.831158, 0.294279, 1.      ],
       [0.678489, 0.863742, 0.189503, 1.      ],
       [0.845561, 0.887322, 0.099702, 1.      ],
       [0.993248, 0.906157, 0.143936, 1.      ]])

In [48]:
fig, axes = plt.subplots(3, 3, figsize=[12, 12], sharey=True)


for i, theta in enumerate(combo_ds.theta.values):
    theta_ds = combo_ds.sel(theta=theta, voltage=slice(-25, v_f))
    ax = axes[i, :]
    
    color = plt.cm.viridis(np.linspace(0, 1, theta_ds.voltage.size))

    
    ave_se_rhos = []
    ave_se_poiss = []
    
    for j, voltage in enumerate(theta_ds.voltage.values):
        pw_plot_ds = theta_ds.sel(voltage=voltage)

        rho = np.squeeze(pw_plot_ds.rho.values)
        pot = np.squeeze(pw_plot_ds.pot.values)
        x_slices, se_rho, se_pois = get_sheath_edges(rho, pot, slice_width=25, threshold=1.5)
        probe_x = x_slices - 130
        
        
        
        ax[0].plot(probe_x, se_rho, 'x', label=voltage, color=color[j])
        ax[1].plot(probe_x, se_pois, 'x', label=voltage, color=color[j])
        
        ax[0].axhline(np.mean(se_rho), color=color[j], ls='--', lw=0.8)
        ax[1].axhline(np.mean(se_pois), color=color[j], ls='--', lw=0.8)

        ave_se_rhos.append(np.mean(se_rho))
        ave_se_poiss.append(np.mean(se_pois))
    
    
    ax[2].plot(np.float_power(np.abs(theta_ds.voltage - v_f), 0.75), theta_ds.delta, 'x', label=r'$\rho$')
    ax[2].plot(np.float_power(np.abs(theta_ds.voltage - v_f), 0.75), ave_se_rhos, 'x', label=r'$\rho$')
    ax[2].plot(np.float_power(np.abs(theta_ds.voltage - v_f), 0.75), ave_se_poiss, 'x', label=r'$\phi$')
        
    ax[0].set_title(r'$\rho$')
    ax[1].set_title(r'$\phi^{\prime\prime}$')
    ax[2].set_title(r'mean across probe')

    ax[0].legend()
    ax[1].legend()
    ax[2].legend()

    ax[0].set_ylabel(r'$\theta = {}$'.format(theta))
    fig.tight_layout()

### Stuff nicked from analysis_1.ipynb

In [51]:
parser = splopter.parser

dV = parser.getfloat('mks', 'mks_te') #* c.BOLTZMANN / c.ELEM_CHARGE
T_e = parser.getfloat('mks', 'mks_te')
n_0 = parser.getfloat('mks', 'mks_n0')

L = parser.getfloat('rectangle2', 'yhigh') - parser.getfloat('rectangle2', 'ylow')
g = parser.getfloat('rectangle2', 'ylow') - parser.getfloat('rectangle0', 'yhigh') 
lambda_D = lpu.debye_length(T_e, n_0)
theta_p = np.radians(10.0)

print(f"L = {L} L_d \n"
      f"g = {g} L_d \n"
      f"lambda_D = {lambda_D} \n"
      f"theta_p = {theta_p} \n")

L = 300.0 L_d 
g = 60.0 L_d 
lambda_D = 1.6622799720325184e-05 
theta_p = 0.17453292519943295 



In [53]:
print(2.38e-3 / lpu.debye_length(6, 6.4e18))
print(5e-3 / lpu.debye_length(6, 6.4e18))

330.6527573670192
694.6486499307126


In [49]:
def decompose_new_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    y = (a * np.sqrt(np.sin(theta)) * (((L + g) * np.tan(theta)) 
                                       + (L * np.tan(theta_p)) - d_perp))
    x = np.tan(theta) + (2 * np.tan(theta_p))
    return x, y

def decompose_alt_new_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    y = (a * np.sqrt(np.sin(theta)) * (L + g + (((L * np.tan(theta_p)) - d_perp) / np.tan(theta))))
    x = np.cos(theta) / np.sin(theta)
    return x, y

def decompose_2d_box_sheath_exp_param(a, theta, L, g, d_perp, theta_p):
    L_eff = (L/np.cos(theta_p)) - ((d_perp - (g * np.tan(theta))) 
                                   / ((np.cos(theta_p) * np.tan(theta)) + np.sin(theta_p)))
    
    y = a * np.sqrt(np.sin(theta + theta_p)) * L_eff
    x = np.cos(theta + theta_p) / np.sin(theta + theta_p)
    return x, y

def decompose_sheath_exp_param(a, theta, L, g, d_perp=0, theta_p=0):
    y = a * (L + g) * np.sqrt(np.sin(theta))
    x = np.cos(theta) / np.sin(theta)
    return x, y


In [50]:
decomp_modes = [
    None, 
    decompose_sheath_exp_param, 
    decompose_new_sheath_exp_param, 
    decompose_2d_box_sheath_exp_param,
    decompose_alt_new_sheath_exp_param
]
mode_labels = [
    None, 
    (r'$\cot{\theta}$', r'$a\sin^{1/2}{\theta}\cdot[L + g]$'),
    (r'$\tan{\theta} + 2\tan{\theta_p}$', r'$a\sin^{1/2}{\theta}[(L+g)\tan{\theta} + L\tan{\theta_p} - d_{\perp}]$'),
    (r'$\cot{\theta_{tot}}$', r'$a\sin^{1/2}{\theta_{tot}}[L_{eff} + \Delta_0 \cot{\theta_{tot}}]$'),
    (r'$\cot{\theta}$', r'$a\sin^{1/2}{\theta}[L + g + (L\tan{\theta_p} - d_{\perp}) * \cot{\theta}]$'),
]
mode_constants = [
    None,
    ('y_0', 'm'),
    ('m', 'y_0'),
    ('y_0', 'm'),
    ('y_0', 'm'),
]

def plot_decomp(ds, sheath_label='ion_a', mode=1, fit_fl=True, ax=None, kwargs_for_plot={}, 
                kwargs_for_fitplot={}, colour='r', plot_label=None):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = ax.figure
    
    if mode in [1, 2, 3, 4]:
        x, y = decomp_modes[mode](ds[sheath_label], ds['theta_rads'], L, g, 
                                  ds['recession'], ds['theta_p_rads'])
    else:
        raise ValueError('Mode must be 1, 2 or 3')
    
    x = x[~np.isnan(y)]
    y = y[~np.isnan(y)]
    
    if plot_label is None:
        plot_label = sheath_label
    
    ax.errorbar(x, y, yerr=ds[sheath_label]*y, color=colour, label=plot_label, **kwargs_for_plot)
    
    sl_fitter = fts.StraightLineFitter()
    fit_data = sl_fitter.fit(x, y)
    if fit_fl:    
        c1, c2 = mode_constants[mode]
        fit_label = r'$c_1$ = {:.2g}, $c_2$ = {:.2g}'.format(fit_data.get_param(c1), fit_data.get_param(c2))
        ax.plot(*fit_data.get_fit_plottables(), color=colour, label=fit_label, 
                **kwargs_for_fitplot)
        
    ax.set_xlabel(mode_labels[mode][0])
    ax.set_ylabel(mode_labels[mode][1])
#     ax.legend()
    
    return ax, fit_data